<a href="https://colab.research.google.com/github/AntonioPiscopo87/Loyalty_Rewards_Subscription_Model/blob/main/Loyalty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

In [4]:
import os
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

name_of_folder = input("Enter Name of the folder: ")
path = "/content/drive/MyDrive/Loyalty_Rewards/RowData/"+ name_of_folder

# Getting all the file names from the folder
filesnames = os.listdir(path)

# Creating empty dataframe for the consolidation
cols = ["Name","Email","Charge Status"]


consolidated_data = pd.DataFrame(columns=["Name","Email","Charge Status","Month_Year"])

for name in filesnames:
    Month_Year = name[:6]
    #print(Month_Year)
    temp_df = pd.read_csv(path+"/"+name)
    #if len(temp_df.columns)>=23:
        #temp_df.rename(columns = {'Lifetime Amount': 'Lifetime $', 'Pledge Amount': 'Pledge $'}, inplace = True)
    temp_df = temp_df[cols]
    temp_df["Month_Year"] = Month_Year
    temp_df = temp_df[temp_df["Charge Status"] == "Paid"]
    #
    consolidated_data = pd.concat([temp_df,consolidated_data])

data = pd.crosstab(consolidated_data.Email, consolidated_data.Month_Year).T.reset_index()
data.Month_Year = pd.to_datetime(data.Month_Year, format="%b %y")

data.reset_index(drop=True, inplace=True)


startDate = min(data.Month_Year)
endDate = max(data.Month_Year)

cur_date = startDate
end = endDate
inData_date = list(data.Month_Year)
actual_date = []
while cur_date <= end:
    actual_date.append(cur_date)
    cur_date += relativedelta(months=1)


missing_date = [val for val in actual_date if val not in inData_date]

data = data.set_index("Month_Year").T

if len(missing_date)>0:
    for d in missing_date:
        data[d] = 0
else:
    pass

data = data.T.reset_index()
data = data.sort_values(by='Month_Year')

def date_month(x):
    return x.strftime("%b-%Y")
data.Month_Year = data.Month_Year.apply(date_month)
data.set_index("Month_Year", inplace = True)

rolling_values = [x for x in range(3,len(data.iloc[:,1]),3)]

Loyalty_Reward = []


for column in tqdm(data):
    #abc[column]
    temp_list = []
    bucket = []
    temp_list.append(column)
    for roll in rolling_values:
        if any(list(data[column].rolling(roll).sum()==roll)):
            bucket.append(str(roll)+" Months")
        else:
            bucket.append("")
    try:
        last_subscription = [i for i in bucket if i][-1]
    except:
        last_subscription = ""
    temp_list.append(bucket)
    temp_list.append(last_subscription)
    Loyalty_Reward.append(temp_list)

Loyalty_Reward_Buckets = pd.DataFrame(Loyalty_Reward).rename(columns={0:"Email", 1:"All Bucket", 2:"Loyalty Reward"})
Loyalty_Reward_Buckets = pd.merge(Loyalty_Reward_Buckets, consolidated_data[["Email","Name"]].drop_duplicates(['Email']),
         on='Email')[["Name","Email","All Bucket","Loyalty Reward"]]
Loyalty_Reward_Buckets.to_csv(str(name_of_folder)+"_Loyalty_Reward_Buckets.csv", index=False)
print("Done")



Enter Name of the folder: Archvillain


4872it [00:01, 2950.58it/s]

Done


In [5]:
files.download(str(name_of_folder)+"_Loyalty_Reward_Buckets.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>